##### 例子: Web Downloads

In [22]:
import os
import time
import sys
import requests

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()
BASE_URL = 'http://flupy.org/data/flags'
DEST_DIR = 'downloads/'


def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content


def show(text):
    print(text, end=' ')
    sys.stdout.flush()


def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

########################


def main(download_func):
    t0 = time.time()
    count = download_func(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))

###### 顺序下载

In [23]:
def download_sequentially(cc_list):
    res = []
    for cc in sorted(cc_list):
        res.append(download_one(cc))
    return len(res)

In [24]:
main(download_sequentially)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 flags downloaded in 2.56s


###### 线程池下载

In [26]:
from concurrent import futures
MAX_WORKERS = 20


def download_concurrently(cc_list):
    """该函数没有显式的调用Future
    """
    num_workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(num_workers) as executor:
        res = executor.map(download_one, sorted(cc_list))
    return len(list(res))

In [27]:
main(download_concurrently)

ET EG BDMX TR DE VN CN CD RU ID PH PK  FR NG US IN BR JP IR 
20 flags downloaded in 0.18s


###### 显式地调用Furture对象的并发版本

In [29]:
def download_concurrently_v2(cc_list):
    num_workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(num_workers) as executor:
        
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {} : {}'
            print(msg.format(cc, future))
        
        results = []
        for future in futures.as_completed(to_do):
            res = future.result()
            msg = '{} result : {!r}'
            print(msg.format(future, res))
            results.append(res)
    
    return len(results)      

In [30]:
main(download_concurrently_v2)

Scheduled for BD : <Future at 0x7f41963595c0 state=running>
Scheduled for BR : <Future at 0x7f4196f927f0 state=running>
Scheduled for CD : <Future at 0x7f4196f9f8d0 state=running>
Scheduled for CN : <Future at 0x7f41963509e8 state=running>
Scheduled for DE : <Future at 0x7f41964cb940 state=running>
Scheduled for EG : <Future at 0x7f41963d17f0 state=running>
Scheduled for ET : <Future at 0x7f4196fc3550 state=running>
Scheduled for FR : <Future at 0x7f4196f9d048 state=running>
Scheduled for ID : <Future at 0x7f4196d8a780 state=running>
Scheduled for IN : <Future at 0x7f4196d8afd0 state=running>
Scheduled for IR : <Future at 0x7f4194290668 state=running>
Scheduled for JP : <Future at 0x7f41942970f0 state=running>
Scheduled for MX : <Future at 0x7f4194297ac8 state=running>
Scheduled for NG : <Future at 0x7f41942797b8 state=running>
Scheduled for PH : <Future at 0x7f4194227240 state=running>
Scheduled for PK : <Future at 0x7f4194227cc0 state=running>
Scheduled for RU : <Future at 0x7f419423

<center>关于Python中的多线程</center>

Every blocking I/O function in the Python standard library **releases the GIL**, allowing other threads to run. <br>
The time.sleep() function also releases the GIL. <br>
Therefore, Python threads are per‐fectly usable in I/O-bound applications, despite the GIL.

##### Concurrent.futures 的若干功能

###### Executor.map

In [4]:
from time import sleep, strftime
from concurrent import futures

def display(*args):
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)

def loiter(n):
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t' * n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t' * n, n))
    return n * 10

In [7]:
def main():
    display('Script starting.')
    executor = futures.ThreadPoolExecutor(max_workers=4)
    results = executor.map(loiter, range(8))
    display('result:', results)
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {} : {}'.format(i, result))

In [8]:
main()

[09:57:19] Script starting.
[09:57:19] loiter(0): doing nothing for 0s...
[09:57:19] loiter(0): done.
[09:57:19] 	loiter(1): doing nothing for 1s...
[09:57:19][09:57:19] 			loiter(3): doing nothing for 3s...
 		loiter(2): doing nothing for 2s...
[09:57:19] result: <generator object Executor.map.<locals>.result_iterator at 0x7ff0b7533fc0>
[09:57:19] Waiting for individual results:
[09:57:19] result 0 : 0
[09:57:19] 				loiter(4): doing nothing for 4s...
[09:57:20] 	loiter(1): done.
[09:57:20] 					loiter(5): doing nothing for 5s...
[09:57:20] result 1 : 10
[09:57:21] 		loiter(2): done.
[09:57:21] 						loiter(6): doing nothing for 6s...
[09:57:21] result 2 : 20
[09:57:22] 			loiter(3): done.
[09:57:22] 							loiter(7): doing nothing for 7s...
[09:57:22] result 3 : 30
[09:57:23] 				loiter(4): done.
[09:57:23] result 4 : 40
[09:57:25] 					loiter(5): done.
[09:57:25] result 5 : 50
[09:57:27] 						loiter(6): done.
[09:57:27] result 6 : 60
[09:57:29] 							loiter(7): done.
[09:57:29] re

<center> Executor.map 的特点</center>

1. 它返回一个生成器对象, 里面**按调用的顺序储存了运行结果**. 因此如果前面的调用耗时比较长, 生成器一开始可能阻塞

###### Error Handling